In [1]:
import numpy as np

In [2]:
def num_veces(search_string, string):
    array_string = string.split(" ")
    cuenta = 0
    for sub_string in array_string:
        if sub_string == search_string:
            cuenta += 1
    return cuenta


def array_palabras(corpus, vocabulario):
    diccionario = {word: [0,[]] for word in vocabulario}
    for id_documento in range(len(corpus)):
        for word in vocabulario:
            if num_veces(word, corpus[id_documento]) > 0:
                diccionario[word][1].append(id_documento+1)
            diccionario[word][0] = len(diccionario[word][1])
    return diccionario

def palabra_en_doc(corpus, palabra):
    lista = np.zeros(len(corpus), dtype=int) 
    for id_documento in range(len(corpus)):
        lista[id_documento] = num_veces(palabra, corpus[id_documento])
    return lista

terminos = [
    "carro",
    "compra",
    "industrial",
    "libro",
    "máquina",
    "mecánica",
    "nuevo",
    "usado",
    "venta",
]

documentos = [
    "venta carro usado",
    "venta libro usado venta libro nuevo",
    "compra máquina industrial máquina mecánica",
    "máquina venta usado",
    "libro máquina industrial nuevo"
]

array_palabras(documentos, terminos)

{'carro': [1, [1]],
 'compra': [1, [3]],
 'industrial': [2, [3, 5]],
 'libro': [2, [2, 5]],
 'máquina': [3, [3, 4, 5]],
 'mecánica': [1, [3]],
 'nuevo': [2, [2, 5]],
 'usado': [3, [1, 2, 4]],
 'venta': [3, [1, 2, 4]]}

In [3]:
np.zeros((1, len([1,2])))

array([[0., 0.]])

In [4]:
#def tf():
for word in terminos:
    print(word, palabra_en_doc(documentos, word))


carro [1 0 0 0 0]
compra [0 0 1 0 0]
industrial [0 0 1 0 1]
libro [0 2 0 0 1]
máquina [0 0 2 1 1]
mecánica [0 0 1 0 0]
nuevo [0 1 0 0 1]
usado [1 1 0 1 0]
venta [1 2 0 1 0]


In [5]:
def num_docs_word(palabra, documentos):
    df = 0
    for id_documento in range(len(documentos)):
        if num_veces(palabra,documentos[id_documento]) > 0:
            df += 1
    return df
    


In [6]:
num_docs_word(terminos[2], documentos)

2

In [7]:
lista_palabras = np.zeros(len(documentos))
lista_palabras

array([0., 0., 0., 0., 0.])

In [8]:
def tf_idf_row(documentos, palabra):
    lista_palabras = np.zeros((len(documentos)))
    N = len(documentos)
    for id_documento in range(len(documentos)):
        idf = (N / num_docs_word(palabra,documentos))
        tf = num_veces(palabra,documentos[id_documento])
        lista_palabras[id_documento] =  tf * idf
        
    return lista_palabras
tf_idf = np.zeros((len(terminos), len(documentos)))
for id_word in range(len(terminos)):
    tf_idf[id_word] = tf_idf_row(documentos, terminos[id_word])
    #print(tf_idf)
    #print("-----------------------")
    #idf = np.log10(1 + palab)

In [9]:
tf_idf

array([[5.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 5.        , 0.        , 0.        ],
       [0.        , 0.        , 2.5       , 0.        , 2.5       ],
       [0.        , 5.        , 0.        , 0.        , 2.5       ],
       [0.        , 0.        , 3.33333333, 1.66666667, 1.66666667],
       [0.        , 0.        , 5.        , 0.        , 0.        ],
       [0.        , 2.5       , 0.        , 0.        , 2.5       ],
       [1.66666667, 1.66666667, 0.        , 1.66666667, 0.        ],
       [1.66666667, 3.33333333, 0.        , 1.66666667, 0.        ]])

In [10]:
import pandas as pd

In [11]:
pd.DataFrame(tf_idf).to_csv("tf_idf_numeros.csv", index=False, header=False)

In [47]:
def tf_idf_row(documentos, palabra):
    N = len(documentos)
    df = num_docs_word(palabra, documentos)
    idf = np.log10(N / df) if df > 0 else 0

    lista_palabras = np.zeros(len(documentos))
    for i, doc in enumerate(documentos):
        tf = np.log10(1+ num_veces(palabra, doc))
        lista_palabras[i] = tf * idf

    return lista_palabras

tf_idf = np.zeros((len(terminos), len(documentos)))
for id_word, termino in enumerate(terminos):
    tf_idf[id_word] = tf_idf_row(documentos, termino)

In [48]:
def tf_idf_query_col(documento_query, documentos, palabras):
    N = len(documentos)
    

    lista_docs = np.zeros(len(palabras))
    for i, palabra in enumerate(palabras):
        tf = np.log10(1+num_veces(palabra, documento_query))
        df = num_docs_word(palabra, documentos)
        idf = np.log10(N / df) if df > 0 else 0
        lista_docs[i] = tf * idf

    return lista_docs

In [49]:
tf_idf_query = np.zeros((len(terminos), 1))
new_doc = "compra máquina helado"
tf_idf_query  = tf_idf_query_col(new_doc, documentos, terminos)

In [50]:
tf_idf_query 

array([0.        , 0.21041094, 0.        , 0.        , 0.06678313,
       0.        , 0.        , 0.        , 0.        ])

In [51]:
tf_idf = np.zeros((len(terminos), len(documentos)))
for id_word, termino in enumerate(terminos):
    tf_idf[id_word] = tf_idf_row(documentos, termino)

In [52]:
tf_idf

array([[0.21041094, 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.21041094, 0.        , 0.        ],
       [0.        , 0.        , 0.11979188, 0.        , 0.11979188],
       [0.        , 0.18986564, 0.        , 0.        , 0.11979188],
       [0.        , 0.        , 0.10584875, 0.06678313, 0.06678313],
       [0.        , 0.        , 0.21041094, 0.        , 0.        ],
       [0.        , 0.11979188, 0.        , 0.        , 0.11979188],
       [0.06678313, 0.06678313, 0.        , 0.06678313, 0.        ],
       [0.06678313, 0.10584875, 0.        , 0.06678313, 0.        ]])

In [53]:
tf_idf_docs = tf_idf

In [56]:
tf_idf_docs

array([[0.21041094, 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.21041094, 0.        , 0.        ],
       [0.        , 0.        , 0.11979188, 0.        , 0.11979188],
       [0.        , 0.18986564, 0.        , 0.        , 0.11979188],
       [0.        , 0.        , 0.10584875, 0.06678313, 0.06678313],
       [0.        , 0.        , 0.21041094, 0.        , 0.        ],
       [0.        , 0.11979188, 0.        , 0.        , 0.11979188],
       [0.06678313, 0.06678313, 0.        , 0.06678313, 0.        ],
       [0.06678313, 0.10584875, 0.        , 0.06678313, 0.        ]])

In [57]:
tf_idf_query

array([0.        , 0.21041094, 0.        , 0.        , 0.06678313,
       0.        , 0.        , 0.        , 0.        ])

In [ ]:
from numpy.linalg import norm
import numpy as np

query_vec = tf_idf_query.flatten()

similaridades = []
for col in tf_idf_docs.T:  
    sim = np.dot(query_vec, col) / (norm(query_vec) * norm(col))
    similaridades.append(sim)

similaridades = np.array(similaridades)

print(similaridades)

[0.         0.         0.68852156 0.17466089 0.09268924]


In [59]:
query_vec

array([0.        , 0.21041094, 0.        , 0.        , 0.06678313,
       0.        , 0.        , 0.        , 0.        ])

In [61]:
mask = tf_idf_query.flatten() > 0

scores = []
for col in tf_idf_docs.T:  
    score = np.sum(col[mask])  
    scores.append(score)

scores = np.array(scores)

top5_idx = np.argsort(scores)[::-1][:5]
top5_scores = scores[top5_idx]

print("documentos ordenados:", top5_idx)
print("Scores ", top5_scores)

documentos ordenados: [2 4 3 1 0]
Scores  [0.31625969 0.06678313 0.06678313 0.         0.        ]
